# MNIST in PyTorch

In [17]:
import os

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

***

## Define model etc.

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [4]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

***

## Settings

In [13]:
batch_size=64
test_batch_size=128
data_dir = os.path.join(os.getcwd(), 'data')
lr = 0.1
momentum = 0.5
epochs = 10

In [6]:
use_cuda = torch.cuda.is_available()
print('Using GPU...' if use_cuda else 'NOT using GPU...')

device = torch.device('cuda:0' if use_cuda else 'cpu')

Using GPU...


In [7]:
seed = 1
torch.manual_seed(seed)

***

## Get dataset

In [8]:
# Since torchvision v0.2.1
datasets.MNIST(data_dir, download=True)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train

In [9]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        data_dir, train=True, download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
    batch_size=batch_size, shuffle=True, num_workers=2)

In [10]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        data_dir, train=False, 
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
    batch_size=test_batch_size, shuffle=True, num_workers=2)

***

## Start training

In [11]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [16]:
for epoch in range(0, epochs):
    train(500, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.173539
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.110024
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.200376
Train Epoch: 1 [45000/60000 (75%)]	Loss: 0.167588

Test set: Average loss: 0.0384, Accuracy: 9871/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.151616
Train Epoch: 2 [15000/60000 (25%)]	Loss: 0.066877
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.099042
Train Epoch: 2 [45000/60000 (75%)]	Loss: 0.106680

Test set: Average loss: 0.0429, Accuracy: 9854/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.059298
Train Epoch: 3 [15000/60000 (25%)]	Loss: 0.103964
Train Epoch: 3 [30000/60000 (50%)]	Loss: 0.032571
Train Epoch: 3 [45000/60000 (75%)]	Loss: 0.015689

Test set: Average loss: 0.0384, Accuracy: 9874/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.048680
Train Epoch: 4 [15000/60000 (25%)]	Loss: 0.231229
Train Epoch: 4 [30000/60000 (50%)]	Loss: 0.110270
Train Epoch: 4 [45000/60000 (75%)]	Loss: 0.026107

Test set: Average loss: 0.0385, Accu

***

## Save trained model

In [15]:
torch.save(model.state_dict(), 'mnist.pt')